In [ ]:
from pyimagesearch.face_blurring import anonymize_face_pixelate
from pyimagesearch.face_blurring import anonymize_face_simple
from helper import draw
import numpy as np
import cv2
import os

image_path = "./images/0.jpeg"
face = "./face_detector"
# face blurring/anonymizing method, choices=["simple", "pixelated"],
method = "pixelated"
# blocks for the pixelated blurring method
blocks = 20
# minimum probability to filter weak detections
confidence_threshold = 0.5

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([face, "deploy.prototxt"])
weightsPath = os.path.sep.join([face,
	"res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

# load the input image from disk, clone it, and grab the image spatial
# dimensions
image = cv2.imread(image_path)
orig = image.copy()
(h, w) = image.shape[:2]

# construct a blob from the image
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
	(104.0, 177.0, 123.0))

# pass the blob through the network and obtain the face detections
print("[INFO] computing face detections...")
net.setInput(blob)
detections = net.forward()

# loop over the detections
for i in range(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with the
	# detection
	confidence = detections[0, 0, i, 2]

	# filter out weak detections by ensuring the confidence is greater
	# than the minimum confidence
	if confidence > confidence_threshold:
		# compute the (x, y)-coordinates of the bounding box for the
		# object
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")

		# extract the face ROI
		face = image[startY:endY, startX:endX]
		
        # check to see if we are applying the "simple" face blurring
		# method
		if method == "simple":
			face = anonymize_face_simple(face, factor=3.0)
		# otherwise, we must be applying the "pixelated" face
		# anonymization method
		else:
			face = anonymize_face_pixelate(face, blocks=blocks)

		# store the blurred face in the output image
		image[startY:endY, startX:endX] = face

# display the original image and the output image with the blurred
# face(s) side by side
output = np.hstack([orig, image])
draw("Output", output)


In [ ]:
from pyimagesearch.face_blurring import anonymize_face_pixelate
from pyimagesearch.face_blurring import anonymize_face_simple
from imutils.video import FileVideoStream
from helper import draw
import numpy as np
import imutils
import time
import cv2
import os

video_path = "./video/1.mp4"
face = "./face_detector"
# face blurring/anonymizing method, choices=["simple", "pixelated"],
method = "pixelated"
# blocks for the pixelated blurring method
blocks = 20
# minimum probability to filter weak detections
confidence_threshold = 0.5

# load our serialized face detector model from disk
print("[INFO] loading face detector model...")
prototxtPath = os.path.sep.join([face, "deploy.prototxt"])
weightsPath = os.path.sep.join([face,
	"res10_300x300_ssd_iter_140000.caffemodel"])
net = cv2.dnn.readNet(prototxtPath, weightsPath)

# initialize the video stream and allow the camera sensor to warm up
print("[INFO] starting video stream...")
vs = FileVideoStream(path=video_path).start()
time.sleep(2.0)

while vs.more():
	# grab the frame from the threaded video stream and resize it
	# to have a maximum width of 400 pixels
    frame = vs.read()
    frame = imutils.resize(frame, width=400)

	# grab the dimensions of the frame and then construct a blob
	# from it
    (h, w) = frame.shape[:2]
    blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300),
		(104.0, 177.0, 123.0))

	# pass the blob through the network and obtain the face detections
    net.setInput(blob)
    detections = net.forward()
	
    for i in range(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the detection
        confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the confidence is
		# greater than the minimum confidence
        if confidence > confidence_threshold:
			# compute the (x, y)-coordinates of the bounding box for
			# the object
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

			# extract the face ROI
            face = frame[startY:endY, startX:endX]

			# check to see if we are applying the "simple" face
			# blurring method
            if method == "simple":
                face = anonymize_face_simple(face, factor=3.0)
			# otherwise, we must be applying the "pixelated" face
			# anonymization method
            else:
                face = anonymize_face_pixelate(face, blocks=blocks)

			# store the blurred face in the output image
            frame[startY:endY, startX:endX] = face

    draw("Frame", frame)

vs.stop()